In [1]:
from dataset_io import *
from classifiers import *
from algorithm import Algorithm, Predictions
from tqdm import tqdm

In [2]:
#np.save("cross_validated_scenes",cross_validated_scenes())
cross_validated_scenes = np.load("cross_validated_scenes.npy", allow_pickle=True)

### Classifiers evaluation

In [3]:
def evaluate_classifiers():
    grad_metrics=[]
    color_metrics=[]

    for fold in tqdm(cross_validated_scenes):
        train_scenes, test_scenes, fold_name = fold
        print(fold_name)
        acc, prec, rec, f1, _ = classify_gradient(train_scenes, test_scenes, fold_name, True)
        grad_metrics.append([acc, prec, rec, f1])
        res = classify_colors(train_scenes, test_scenes, fold_name, True)
        color_metrics.append(res)

    np.save("color_metrics", color_metrics)
    np.save("grad_metrics", grad_metrics)
    
#evaluate_classifiers()

In [4]:
color_metrics = np.load("color_metrics.npy")
grad_metrics = np.load("grad_metrics.npy")
grad_metrics_avg = np.mean(grad_metrics, 0)
color_metrics_avg = np.mean(color_metrics, 0)
print("color_metrics_avg")
print(color_metrics_avg)
print("grad_metrics_avg")
print(grad_metrics_avg)

color_metrics_avg
[[0.85106473 0.76691145 0.81843552 0.79046445]
 [0.89788648 0.83362693 0.8696812  0.85032493]
 [0.88802002 0.79463581 0.90192808 0.84422685]]
grad_metrics_avg
[0.98093372 0.9983972  0.92963724 0.95946163]


### Algorithms evaluation

In [5]:
def evaluate_algorithm():
    overlaps=[]
    color_means=[]
    grad_means=[]
    color_counts=[]
    grad_counts=[]
    color_stds=[]
    grad_stds=[]

    for fold in cross_validated_scenes:
        train_scenes, test_scenes, fold_name = fold
        print(fold_name)
        alg = Algorithm(train_scenes, test_scenes, fold_name)
        res_path = alg.run()
        color_mean, grad_mean, color_count, grad_count, color_std, grad_std = alg.get_norm_coeffs()
        color_means.append(color_mean)
        grad_means.append(grad_mean)
        color_counts.append(color_count)
        grad_counts.append(grad_count)
        color_stds.append(color_std)
        grad_stds.append(grad_std)
        
        print(color_mean, grad_mean, color_count, grad_count, color_std, grad_std)
        agg_overlaps = Predictions(res_path, test_scenes).draw_and_calc()
        print("agg_overlaps",agg_overlaps)
        overlaps.append(agg_overlaps)

    color_mean = np.average(color_means, weights=np.divide(color_counts,len(color_counts)))
    grad_mean = np.average(grad_means, weights=np.divide(grad_counts,len(grad_counts)))
    color_std = np.mean(color_stds)
    grad_std = np.mean(grad_stds)
    norm_coeffs = [color_mean, grad_mean, color_std, grad_std]
    np.save(PROJECT_PATH+"results/agg_overlaps_weighted", np.mean(overlaps, 0))
    np.save(PROJECT_PATH+"results/norm_coeffs_weighted", norm_coeffs)
    
evaluate_algorithm()

  0%|          | 0/471 [00:00<?, ?it/s]

fold_0


  0%|          | 2/471 [00:00<00:33, 13.91it/s]

0.454972486252307 0.04673774881042295 12325599 12325599 0.08981628797604502 0.10267146950033552
draw_and_calc on  ['weighted_sum', 'weighted_sum_norm']


  0%|          | 0/577 [00:00<?, ?it/s]

...

agg_overlaps [0.5912634431046445, 0.5987990873577744]
fold_3


  0%|          | 2/555 [00:00<00:46, 11.83it/s]

0.4621282024793426 0.04723494147321165 14093589 14093589 0.08829061476349989 0.09533258673115663
draw_and_calc on  ['weighted_sum', 'weighted_sum_norm']


100%|██████████| 555/555 [00:23<00:00, 23.14it/s]

agg_overlaps [0.6660423733093461, 0.6697852427546839]


In [8]:
np.load(PROJECT_PATH+"results/agg_overlaps_weighted.npy")

array([0.67709032, 0.68211362])

In [7]:
np.load(PROJECT_PATH+"results/norm_coeffs_norm.npy")

array([0.46017551, 0.04681373, 0.08855855, 0.09643027])

color, grad, max, min, sum, max_norm, min_norm, sum_norm, weighted_sum, weighted_sum_norm

0.49258194, 0.59860443, 0.59728843, 0.68011496, 0.68655563, 0.59936776, 0.62096155, 0.69024978, 0.67709032, 0.68211362

color_mean, grad_mean, color_std, grad_std

0.46017551, 0.04681373, 0.08855855, 0.09643027

TODO change prob thresh and run all